# 1. 데이터 로딩 및 전처리

In [ ]:
import pandas as pd
file_path = "./data/jung-gu.xlsx"
data = pd.read_excel(file_path)
data.head()

In [ ]:
# 결측값 확인
missing_values = data.isnull().sum()

# 결측값이 있는 열 출력
missing_values[missing_values > 0]

In [ ]:
# 보간법을 사용하여 결측치 처리
data_interpolated = data.interpolate(method='linear')

# 결측치 처리 후의 처음 5개 행을 보여줌
data_interpolated.head()


In [ ]:
# 필요한 변수만 선택
selected_columns = ['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']
data_selected = data_interpolated[selected_columns]

# 선택된 변수의 처음 5개 행을 보여줌
data_selected.head()

In [ ]:
from sklearn.preprocessing import PowerTransformer
import pandas as pd

# 'PM25' 컬럼을 제외한 나머지 컬럼 선택
selected_columns_without_PM25 = [col for col in selected_columns if col != 'PM25']
data_to_transform = data_selected[selected_columns_without_PM25]

# PowerTransformer 객체 생성
power_transformer = PowerTransformer()

# PowerTransformer를 사용하여 선택한 컬럼만 변환
data_power_transformed = power_transformer.fit_transform(data_to_transform)

# 변환된 데이터를 데이터프레임으로 변환
data_power_transformed_df_without_PM25 = pd.DataFrame(data_power_transformed, columns=selected_columns_without_PM25)

# 원래의 'PM25' 컬럼과 변환된 데이터를 합침
data_power_transformed_df = pd.concat([data_power_transformed_df_without_PM25, data_selected['PM25'].reset_index(drop=True)], axis=1)

# 처음 5개 행을 보여줌
data_power_transformed_df.head()


# 2.모델 학습

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# 시간 윈도우 길이 설정
window_length = 10

# 입력 특성 및 타깃 레이블 생성 함수
def create_sequences(data, target_column, window_length):
    sequences = []
    target = []
    for i in range(len(data) - window_length):
        sequences.append(data[i:i + window_length].values)
        target.append(data[target_column].iloc[i + window_length])
    return np.array(sequences), np.array(target)

# 입력 특성 및 타깃 레이블 생성
X, y = create_sequences(data_power_transformed_df, target_column='PM25', window_length=window_length)

# 학습, 검증, 테스트 세트로 분할
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 데이터 크기 확인
X_train.shape, X_val.shape, X_test.shape


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# MLP 모델 구성
def build_mlp_model(input_shape):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 모델 생성
mlp_model = build_mlp_model(input_shape=X_train.shape[1:])

# 모델 학습
mlp_history = mlp_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, verbose=1)

# MLP 모델 저장
mlp_model.save('./model/mlp_model.h5')


In [ ]:
from tensorflow.keras.layers import LSTM

# LSTM 모델 구성 함수
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 모델 생성 및 학습
lstm_model = build_lstm_model(input_shape=X_train.shape[1:])
lstm_history = lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# 모델 저장
lstm_model.save('./model/lstm_model.h5')


In [ ]:
from tensorflow.keras.layers import GRU

# GRU 모델 구성 함수
def build_gru_model(input_shape):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=input_shape),
        GRU(32),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# GRU 모델 생성 및 학습
gru_model = build_gru_model(input_shape=X_train.shape[1:])
gru_history = gru_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# GRU 모델 저장
gru_model.save('./model/gru_model.h5')


In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D

def build_cnn_model(input_shape):
    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# CNN 모델 생성 및 학습
cnn_model = build_cnn_model(input_shape=X_train.shape[1:])
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# CNN 모델 저장
cnn_model.save('./model/cnn_model.h5')


In [ ]:
from tensorflow.keras.layers import SimpleRNN

def build_simplernn_model(input_shape):
    model = Sequential([
        SimpleRNN(64, return_sequences=True, input_shape=input_shape),
        SimpleRNN(32),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# SimpleRNN 모델 생성 및 학습
simplernn_model = build_simplernn_model(input_shape=X_train.shape[1:])
simplernn_history = simplernn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# SimpleRNN 모델 저장
simplernn_model.save('./model/simplernn_model.h5')


In [ ]:
# Temporal Convolutional Network (TCN) 기반 모델

from tensorflow.keras.layers import Input, Conv1D, Activation, BatchNormalization, SpatialDropout1D, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def residual_block(x, filters, kernel_size, dilation_rate):
    prev_x = x
    for k in range(2):
        x = Conv1D(filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(x)
        x = Activation('relu')(x)
        x = BatchNormalization()(x)
        x = SpatialDropout1D(0.2)(x)

    # Match the dimensions of the residual block and input
    if x.shape[-1] != prev_x.shape[-1]:
        prev_x = Conv1D(filters=filters, kernel_size=1)(prev_x)
        
    out = x + prev_x
    return out

def build_tcn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = inputs
    for _ in range(3): # You can adjust the number of layers
        x = residual_block(x, filters=64, kernel_size=3, dilation_rate=1)
    x = Lambda(lambda x: x[:, -1, :])(x) # Get the last time step
    output = Dense(1)(x)

    model = Model(inputs, output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

tcn_model = build_tcn_model(input_shape=X_train.shape[1:])
tcn_history = tcn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# TCN 모델 저장
tcn_model.save('./model/tcn_model.h5')


In [ ]:
from spektral.layers import GCNConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense

def build_gcn_model(input_shape):
    # 그래프 형식의 입력
    X_input = Input(shape=(input_shape[0], input_shape[1]))
    A_input = Input(shape=(input_shape[0], input_shape[0]))  # 인접 행렬

    # 그래프 컨볼루션 레이어
    x = GCNConv(64, activation='relu')([X_input, A_input])
    x = GCNConv(32, activation='relu')([x, A_input])

    # 완전 연결 레이어
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)

    model = Model(inputs=[X_input, A_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')

    return model

# 모델 생성 및 학습
gcn_model = build_gcn_model(input_shape=(num_nodes, num_features))
gcn_model.fit([X_train, A_train], y_train, validation_data=([X_val, A_val], y_val), epochs=10, batch_size=32)


# 3. 실시간 미세먼지 예측 대시보드

In [ ]:
import tensorflow as tf
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 모델 불러오기
global graph_mlp, session_mlp
graph_mlp = tf.Graph()
with graph_mlp.as_default():
    session_mlp = tf.compat.v1.Session()
    with session_mlp.as_default():
        global mlp_model
        mlp_model = load_model('./model/mlp_model.h5')

global graph_lstm, session_lstm
graph_lstm = tf.Graph()
with graph_lstm.as_default():
    session_lstm = tf.compat.v1.Session()
    with session_lstm.as_default():
        global lstm_model
        lstm_model = load_model('./model/lstm_model.h5')

global graph_gru, session_gru
graph_gru = tf.Graph()
with graph_gru.as_default():
    session_gru = tf.compat.v1.Session()
    with session_gru.as_default():
        global gru_model
        gru_model = load_model('./model/gru_model.h5')
        
# CNN 모델 불러오기
global graph_cnn, session_cnn
graph_cnn = tf.Graph()
with graph_cnn.as_default():
    session_cnn = tf.compat.v1.Session()
    with session_cnn.as_default():
        global cnn_model
        cnn_model = load_model('./model/cnn_model.h5')

# SimpleRNN 모델 불러오기
global graph_simplernn, session_simplernn
graph_simplernn = tf.Graph()
with graph_simplernn.as_default():
    session_simplernn = tf.compat.v1.Session()
    with session_simplernn.as_default():
        global simplernn_model
        simplernn_model = load_model('./model/simplernn_model.h5')


stations = [
    {'name': '종로구'},
    {'name': '강남구'},
    {'name': '중구'}
    # ... 계속 추가
]

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1(children='PM2.5 Prediction Dashboard', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='station-dropdown',
        options=[{'label': station['name'], 'value': station['name']} for station in stations],
        value='중구'
    ),
    dcc.Graph(id='graph-content'),
    dcc.Interval(id='interval-component', interval=60 * 60 * 1000, n_intervals=0)
])



def preprocess_data(data):
    for i in range(1, len(data)):
        for j in range(6):
            if data[i, j] is None and data[i-1, j] is not None:
                data[i, j] = data[i-1, j]
    
    # 스케일링
    scaler = MinMaxScaler()
    preprocessed_data = scaler.fit_transform(data)
        # NaN 값을 검사하고 0으로 대체
    preprocessed_data = np.nan_to_num(preprocessed_data)
    
    return preprocessed_data

@app.callback(
    Output('graph-content', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('station-dropdown', 'value')]
)


def update_graph(n, value):
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    current_time = datetime.now()
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    time_data = [current_time - timedelta(hours=i) for i in range(10, 0, -1)]
    # 사용자 정의 눈금 값과 레이블 생성 (시간만 표시)
    tickvals = []
    ticktext = []
    for i, time in enumerate(time_data):
        tickvals.append(time)
        ticktext.append(time.strftime("%H:%M"))  # 연도, 월, 일 제거
        
    # fig 객체 생성
    fig = go.Figure()
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="PM2.5 Concentration (μg/m³)"
    )

    # 연도, 월, 일을 상단 좌측에 표시 (예: 2023-08-21)
    fig.add_annotation(
        text=current_time.strftime("%Y-%m-%d"),
        xref="paper",
        yref="paper",
        x=0,  # 좌측 정렬
        y=1,  # 상단 정렬
        showarrow=False,
        font=dict(size=14),
      
    )
    
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
    service_key = 'xdBtiCkOLc9TIoPuquKxtR0n+Fk8awNT/qLlOCApNheoK+gweAWhIWzdUCttWKvi0xeI2bM3KCSduGfRhI922w=='


    # 선택된 측정소에 대한 데이터 수집
    params = {
        'serviceKey': service_key,
        'returnType': 'json',
        'numOfRows': '10',
        'pageNo': '1',
        'stationName': value,
        'dataTerm': 'DAILY',
        'ver': '1.0'
    }
    response = requests.get(url, params=params)
    json_object = json.loads(response.content)
    
    # 6개의 특성을 저장할 리스트
    data = []
    for item in json_object['response']['body']['items']:
        values = [
            float(item['pm10Value']) if item['pm10Value'] != "-" else None,
            float(item['pm25Value']) if item['pm25Value'] != "-" else None,
            float(item['o3Value']) if item['o3Value'] != "-" else None,
            float(item['no2Value']) if item['no2Value'] != "-" else None,
            float(item['coValue']) if item['coValue'] != "-" else None,
            float(item['so2Value']) if item['so2Value'] != "-" else None
        ]
        data.append(values)

    # 결측값 처리 및 numpy 배열로 변환
    data = np.array(data)
    data = preprocess_data(data)  # 호출 추가

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 실제 값 준비 (예: 최근 10개의 pm25 값)
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # MLP 모델 예측
    with graph_mlp.as_default():
        with session_mlp.as_default():
            mlp_pred = mlp_model.predict(input_data)

    # LSTM 모델 예측
    with graph_lstm.as_default():
        with session_lstm.as_default():
            lstm_pred = lstm_model.predict(input_data)

    # GRU 모델 예측
    with graph_gru.as_default():
        with session_gru.as_default():
            gru_pred = gru_model.predict(input_data)
    # CNN 모델 예측
    with graph_cnn.as_default():
        with session_cnn.as_default():
            cnn_pred = cnn_model.predict(input_data)

    # SimpleRNN 모델 예측
    with graph_simplernn.as_default():
        with session_simplernn.as_default():
            simplernn_pred = simplernn_model.predict(input_data)
            

    # 예측 결과에서 1시간 뒤의 값을 추출
    mlp_1hour_pred = mlp_pred[0, -1]
    lstm_1hour_pred = lstm_pred[0, -1]
    gru_1hour_pred = gru_pred[0, -1]
    cnn_1hour_pred = cnn_pred[0, -1]
    simplernn_1hour_pred = simplernn_pred[0, -1]
    

    # RMSE 계산 수정
    mlp_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [mlp_1hour_pred]))
    lstm_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [lstm_1hour_pred]))
    gru_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [gru_1hour_pred]))
    cnn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [cnn_1hour_pred]))
    simplernn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [simplernn_1hour_pred]))
    fig.update_layout(annotations=[])


    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 현재 시간부터 10시간 전까지의 시간 데이터 생성
    time_data = [current_time - timedelta(hours=i) for i in range(10)]

    # 현재 시간부터 10시간 전까지의 실제 값 준비
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # 마지막 실제 값
    last_actual_value = actual_data[-1]
    
    # 실제 값의 선 색깔 정의
    actual_line_color = 'rgb(30, 144, 255)'  # Dodger Blue
    # 실제 데이터를 그래프에 추가 (선 색깔을 설정)
    fig.add_trace(go.Scatter(x=time_data, y=actual_data, mode='markers+lines', name=f"실제값 ({value})", line=dict(color=actual_line_color)))

    # 예측 결과와 마지막 실제 값 사이의 점선을 그래프에 추가 (선 색깔을 실제 값의 선 색깔과 동일하게 설정)
    fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, mlp_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, simplernn_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, gru_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, cnn_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    # 예측 마커 추가
    fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[mlp_1hour_pred], mode='markers', name=f"MLP 예측값 (RMSE: {mlp_rmse:.2f})"))
    fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[simplernn_1hour_pred], mode='markers', name=f"RNN 예측값 (RMSE: {simplernn_rmse:.2f})"))
    fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[gru_1hour_pred], mode='markers', name=f"GRU 예측값 (RMSE: {gru_rmse:.2f})"))
    fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[cnn_1hour_pred], mode='markers', name=f"CNN 예측값 (RMSE: {cnn_rmse:.2f})"))


    # 예측 결과를 그래프에 추가 (MSE 값 포함)
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[mlp_1hour_pred], mode='markers', name=f"MLP 예측값 (RMSE: {mlp_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[simplernn_1hour_pred], mode='markers', name=f"RNN 예측값 (RMSE: {simplernn_rmse:.2f})"))
    #fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[lstm_1hour_pred], mode='markers', name=f"LSTM 예측값 (RMSE: {lstm_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[gru_1hour_pred], mode='markers', name=f"GRU 예측값 (RMSE: {gru_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[cnn_1hour_pred], mode='markers', name=f"CNN 예측값 (RMSE: {cnn_rmse:.2f})"))
    #fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[simplernn_1hour_pred], mode='markers', name=f"RNN 예측값 (RMSE: {simplernn_rmse:.2f})"))
    return fig

# 앱 실행
if __name__ == "__main__":
    app.run_server(port=8008, host='0.0.0.0')


In [ ]:
import tensorflow as tf
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 모델 불러오기
global graph_mlp, session_mlp
graph_mlp = tf.Graph()
with graph_mlp.as_default():
    session_mlp = tf.compat.v1.Session()
    with session_mlp.as_default():
        global mlp_model
        mlp_model = load_model('./model/mlp_model.h5')

global graph_lstm, session_lstm
graph_lstm = tf.Graph()
with graph_lstm.as_default():
    session_lstm = tf.compat.v1.Session()
    with session_lstm.as_default():
        global lstm_model
        lstm_model = load_model('./model/lstm_model.h5')

global graph_gru, session_gru
graph_gru = tf.Graph()
with graph_gru.as_default():
    session_gru = tf.compat.v1.Session()
    with session_gru.as_default():
        global gru_model
        gru_model = load_model('./model/gru_model.h5')
        
# CNN 모델 불러오기
global graph_cnn, session_cnn
graph_cnn = tf.Graph()
with graph_cnn.as_default():
    session_cnn = tf.compat.v1.Session()
    with session_cnn.as_default():
        global cnn_model
        cnn_model = load_model('./model/cnn_model.h5')

# SimpleRNN 모델 불러오기
global graph_simplernn, session_simplernn
graph_simplernn = tf.Graph()
with graph_simplernn.as_default():
    session_simplernn = tf.compat.v1.Session()
    with session_simplernn.as_default():
        global simplernn_model
        simplernn_model = load_model('./model/simplernn_model.h5')


stations = [
    {'name': '종로구'},
    {'name': '강남구'},
    {'name': '중구'}
    # ... 계속 추가
]

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1(children='실시간 초미세먼지 예측 대시보드', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='station-dropdown',
        options=[{'label': station['name'], 'value': station['name']} for station in stations],
        value='중구'
    ),
    dcc.Graph(id='graph-content'),
    html.Div([
        html.P('출처: 한국환경공단, 「대기오염정보」', style={'font-size': 'small'}),
    ], style={'position': 'relative', 'bottom': '40px', 'left': '10px'}),

    dcc.Interval(id='interval-component', interval=60 * 60 * 1000, n_intervals=0),
    html.Div([
        html.H3('대시보드 설명:', style={'padding': '8px'}),
        html.P('다양한 머신러닝 모델을 사용하여 실시간으로 PM2.5 농도를 예측함.'),
        html.P('- 예측에 사용된 모델:'),
        html.Ul([
            html.Li('MLP (Multi-Layer Perceptron)'),
            html.Li('LSTM (Long Short-Term Memory)'),
            html.Li('GRU (Gated Recurrent Unit)'),
            html.Li('CNN (Convolutional Neural Network)'),
            html.Li('RNN (Recurrent Neural Network)'),
        ]),
        html.P('- 예측 성능 지표: RMSE (Root Mean Square Error)'),
        html.P('- 예측 결과: 예측 성능이 가장 좋은 모델에 대해서만 출력'),
    ], style={'padding': '20px', 'border': '1px solid #ddd', 'border-radius': '8px', 'margin': '20px'}) ])


def preprocess_data(data):
    for i in range(1, len(data)):
        for j in range(6):
            if data[i, j] is None and data[i-1, j] is not None:
                data[i, j] = data[i-1, j]
    
    # 스케일링
    scaler = MinMaxScaler()
    preprocessed_data = scaler.fit_transform(data)
        # NaN 값을 검사하고 0으로 대체
    preprocessed_data = np.nan_to_num(preprocessed_data)
    
    return preprocessed_data

@app.callback(
    Output('graph-content', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('station-dropdown', 'value')]
)


def update_graph(n, value):
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    current_time = datetime.now()
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    time_data = [current_time - timedelta(hours=i) for i in range(10, 0, -1)]
    # 사용자 정의 눈금 값과 레이블 생성 (시간만 표시)
    tickvals = []
    ticktext = []
    for i, time in enumerate(time_data):
        tickvals.append(time)
        ticktext.append(time.strftime("%H:%M"))  # 연도, 월, 일 제거
        
    # fig 객체 생성
    fig = go.Figure()
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="PM2.5 Concentration (μg/m³)"
    )

    # 연도, 월, 일을 상단 좌측에 표시 (예: 2023-08-21)
    fig.add_annotation(
        text=current_time.strftime("%Y-%m-%d"),
        xref="paper",
        yref="paper",
        x=0,  # 좌측 정렬
        y=1,  # 상단 정렬
        showarrow=False,
        font=dict(size=14),
      
    )
    
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
    service_key = 'xdBtiCkOLc9TIoPuquKxtR0n+Fk8awNT/qLlOCApNheoK+gweAWhIWzdUCttWKvi0xeI2bM3KCSduGfRhI922w=='


    # 선택된 측정소에 대한 데이터 수집
    params = {
        'serviceKey': service_key,
        'returnType': 'json',
        'numOfRows': '10',
        'pageNo': '1',
        'stationName': value,
        'dataTerm': 'DAILY',
        'ver': '1.0'
    }
    response = requests.get(url, params=params)
    json_object = json.loads(response.content)
    
    # 6개의 특성을 저장할 리스트
    data = []
    for item in json_object['response']['body']['items']:
        values = [
            float(item['pm10Value']) if item['pm10Value'] != "-" else None,
            float(item['pm25Value']) if item['pm25Value'] != "-" else None,
            float(item['o3Value']) if item['o3Value'] != "-" else None,
            float(item['no2Value']) if item['no2Value'] != "-" else None,
            float(item['coValue']) if item['coValue'] != "-" else None,
            float(item['so2Value']) if item['so2Value'] != "-" else None
        ]
        data.append(values)

    # 결측값 처리 및 numpy 배열로 변환
    data = np.array(data)
    data = preprocess_data(data)  # 호출 추가

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 실제 값 준비 (예: 최근 10개의 pm25 값)
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # MLP 모델 예측
    with graph_mlp.as_default():
        with session_mlp.as_default():
            mlp_pred = mlp_model.predict(input_data)

    # LSTM 모델 예측
    with graph_lstm.as_default():
        with session_lstm.as_default():
            lstm_pred = lstm_model.predict(input_data)

    # GRU 모델 예측
    with graph_gru.as_default():
        with session_gru.as_default():
            gru_pred = gru_model.predict(input_data)
    # CNN 모델 예측
    with graph_cnn.as_default():
        with session_cnn.as_default():
            cnn_pred = cnn_model.predict(input_data)

    # SimpleRNN 모델 예측
    with graph_simplernn.as_default():
        with session_simplernn.as_default():
            simplernn_pred = simplernn_model.predict(input_data)
            

    # 예측 결과에서 1시간 뒤의 값을 추출
    mlp_1hour_pred = mlp_pred[0, -1]
    lstm_1hour_pred = lstm_pred[0, -1]
    gru_1hour_pred = gru_pred[0, -1]
    cnn_1hour_pred = cnn_pred[0, -1]
    simplernn_1hour_pred = simplernn_pred[0, -1]
    

    # RMSE 계산 수정
    mlp_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [mlp_1hour_pred]))
    lstm_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [lstm_1hour_pred]))
    gru_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [gru_1hour_pred]))
    cnn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [cnn_1hour_pred]))
    simplernn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [simplernn_1hour_pred]))
    fig.update_layout(annotations=[])
    # 모든 모델의 RMSE를 딕셔너리에 저장
    rmse_dict = {
        'MLP': mlp_rmse,
        'LSTM': lstm_rmse,
        'GRU': gru_rmse,
        'CNN': cnn_rmse,
        'SimpleRNN': simplernn_rmse
    }

    # 가장 낮은 RMSE를 가진 모델 찾기
    min_rmse_model = min(rmse_dict, key=rmse_dict.get)
    min_rmse_value = rmse_dict[min_rmse_model]

    # 해당 모델의 예측값 찾기
    pred_dict = {
        'MLP': mlp_1hour_pred,
        'LSTM': lstm_1hour_pred,
        'GRU': gru_1hour_pred,
        'CNN': cnn_1hour_pred,
        'SimpleRNN': simplernn_1hour_pred
    }
    min_rmse_pred = pred_dict[min_rmse_model]

    # 가장 낮은 RMSE를 가진 모델의 예측값만 그래프에 추가
    fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[min_rmse_pred], mode='markers', name=f"{min_rmse_model} 예측값 (RMSE: {min_rmse_value:.2f})"))

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 현재 시간부터 10시간 전까지의 시간 데이터 생성
    time_data = [current_time - timedelta(hours=i) for i in range(10)]

    # 현재 시간부터 10시간 전까지의 실제 값 준비
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # 마지막 실제 값
    last_actual_value = actual_data[-1]
    
    # 실제 값의 선 색깔 정의
    actual_line_color = 'rgb(30, 144, 255)'  # Dodger Blue

    # 실제 데이터를 그래프에 추가 (선 색깔을 설정)
    fig.add_trace(go.Scatter(x=time_data, y=actual_data, mode='markers+lines', name=f"실제값 ({value})", line=dict(color=actual_line_color)))

    # 예측 결과와 마지막 실제 값 사이의 점선을 그래프에 추가 (선 색깔을 실제 값의 선 색깔과 동일하게 설정)
    # 예측 마커와 실측값 마지막 데이터를 점선으로 연결 (MLP 예)
    #fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, mlp_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    #fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, simplernn_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    #fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, gru_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))
    #fig.add_trace(go.Scatter(x=[current_time, current_time + timedelta(hours=1)], y=[last_actual_value, cnn_1hour_pred], mode='lines', line=dict(dash='dash', color=actual_line_color), showlegend=False))

    # 예측 마커 추가
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[mlp_1hour_pred], mode='markers', name=f"MLP 예측값 (RMSE: {mlp_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[simplernn_1hour_pred], mode='markers', name=f"RNN 예측값 (RMSE: {simplernn_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[gru_1hour_pred], mode='markers', name=f"GRU 예측값 (RMSE: {gru_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[cnn_1hour_pred], mode='markers', name=f"CNN 예측값 (RMSE: {cnn_rmse:.2f})"))


    # 예측 결과를 그래프에 추가 (MSE 값 포함)
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[mlp_1hour_pred], mode='markers', name=f"MLP 예측값 (RMSE: {mlp_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[simplernn_1hour_pred], mode='markers', name=f"RNN 예측값 (RMSE: {simplernn_rmse:.2f})"))
    #fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[lstm_1hour_pred], mode='markers', name=f"LSTM 예측값 (RMSE: {lstm_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[gru_1hour_pred], mode='markers', name=f"GRU 예측값 (RMSE: {gru_rmse:.2f})"))
   # fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[cnn_1hour_pred], mode='markers', name=f"CNN 예측값 (RMSE: {cnn_rmse:.2f})"))
    #fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], y=[simplernn_1hour_pred], mode='markers', name=f"RNN 예측값 (RMSE: {simplernn_rmse:.2f})"))
    return fig

# 앱 실행
if __name__ == "__main__":
    app.run_server(port=8008, host='0.0.0.0')


In [ ]:
import tensorflow as tf
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 모델 불러오기
global graph_mlp, session_mlp
graph_mlp = tf.Graph()
with graph_mlp.as_default():
    session_mlp = tf.compat.v1.Session()
    with session_mlp.as_default():
        global mlp_model
        mlp_model = load_model('./model/mlp_model.h5')

global graph_lstm, session_lstm
graph_lstm = tf.Graph()
with graph_lstm.as_default():
    session_lstm = tf.compat.v1.Session()
    with session_lstm.as_default():
        global lstm_model
        lstm_model = load_model('./model/lstm_model.h5')

global graph_gru, session_gru
graph_gru = tf.Graph()
with graph_gru.as_default():
    session_gru = tf.compat.v1.Session()
    with session_gru.as_default():
        global gru_model
        gru_model = load_model('./model/gru_model.h5')
        
# CNN 모델 불러오기
global graph_cnn, session_cnn
graph_cnn = tf.Graph()
with graph_cnn.as_default():
    session_cnn = tf.compat.v1.Session()
    with session_cnn.as_default():
        global cnn_model
        cnn_model = load_model('./model/cnn_model.h5')

# SimpleRNN 모델 불러오기
global graph_simplernn, session_simplernn
graph_simplernn = tf.Graph()
with graph_simplernn.as_default():
    session_simplernn = tf.compat.v1.Session()
    with session_simplernn.as_default():
        global simplernn_model
        simplernn_model = load_model('./model/simplernn_model.h5')


stations = [
    {'name': '종로구'},
    {'name': '강남구'},
    {'name': '중구'}
    # ... 계속 추가
]

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1(children='실시간 초미세먼지 예측 대시보드', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='station-dropdown',
        options=[{'label': station['name'], 'value': station['name']} for station in stations],
        value='중구'
    ),
    dcc.Graph(id='graph-content'),
    html.Div([
        html.P('출처: 한국환경공단, 「대기오염정보」', style={'font-size': 'small'}),
    ], style={'position': 'relative', 'bottom': '40px', 'left': '10px'}),

    dcc.Interval(id='interval-component', interval=60 * 60 * 1000, n_intervals=0),
    

    html.Div([
    html.H3('대시보드 설명', style={'padding': '3px', 'font-weight': 'bold'}),
    html.Div([
        #html.H5('주기 및 예측 범위:'),
        html.P('다양한 머신러닝 모델을 사용하여 1시간 주기로 초미세먼지 농도를 예측')
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 변수: 이산화황(SO2), 일산화탄소(CO), 오존(O3), 이산화질소(NO2), 미세먼지(PM10), 초미세먼지(PM2.5)'])
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 모델: MLP (Multi-Layer Perceptron), RNN (Recurrent Neural Network), LSTM (Long Short-Term Memory), GRU (Gated Recurrent Unit), CNN (Convolutional Neural Network)']),
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 성능 지표: RMSE (Root Mean Square Error)'])
        
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 결과 해석: 예측 성능이 가장 좋은 모델의 예측 값을 출력'])
    ], style={'margin-bottom': '5px'}),

], style={
    'padding': '20px', 
    'border': '1px solid #ddd', 
    'border-radius': '8px', 
    'margin': '20px',
    'box-shadow': '2px 2px 12px #aaa',
    'background-color': '#f9f9f9'
}) ])


def preprocess_data(data):
    for i in range(1, len(data)):
        for j in range(6):
            if data[i, j] is None and data[i-1, j] is not None:
                data[i, j] = data[i-1, j]
    
    # 스케일링
    scaler = MinMaxScaler()
    preprocessed_data = scaler.fit_transform(data)
        # NaN 값을 검사하고 0으로 대체
    preprocessed_data = np.nan_to_num(preprocessed_data)
    
    return preprocessed_data

@app.callback(
    Output('graph-content', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('station-dropdown', 'value')]
)


def update_graph(n, value):
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    current_time = datetime.now()
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    time_data = [current_time - timedelta(hours=i) for i in range(10, 0, -1)]
    # 사용자 정의 눈금 값과 레이블 생성 (시간만 표시)
    tickvals = []
    ticktext = []
    for i, time in enumerate(time_data):
        tickvals.append(time)
        ticktext.append(time.strftime("%H:%M"))  # 연도, 월, 일 제거
        
    # fig 객체 생성
    fig = go.Figure()
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="PM2.5 Concentration (μg/m³)"
    )

    # 연도, 월, 일을 상단 좌측에 표시 (예: 2023-08-21)
    fig.add_annotation(
        text=current_time.strftime("%Y-%m-%d"),
        xref="paper",
        yref="paper",
        x=0,  # 좌측 정렬
        y=1,  # 상단 정렬
        showarrow=False,
        font=dict(size=14),
      
    )
    
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
    service_key = 'xdBtiCkOLc9TIoPuquKxtR0n+Fk8awNT/qLlOCApNheoK+gweAWhIWzdUCttWKvi0xeI2bM3KCSduGfRhI922w=='


    # 선택된 측정소에 대한 데이터 수집
    params = {
        'serviceKey': service_key,
        'returnType': 'json',
        'numOfRows': '10',
        'pageNo': '1',
        'stationName': value,
        'dataTerm': 'DAILY',
        'ver': '1.0'
    }
    response = requests.get(url, params=params)
    json_object = json.loads(response.content)
    
    # 6개의 특성을 저장할 리스트
    data = []
    for item in json_object['response']['body']['items']:
        values = [
            float(item['pm10Value']) if item['pm10Value'] != "-" else None,
            float(item['pm25Value']) if item['pm25Value'] != "-" else None,
            float(item['o3Value']) if item['o3Value'] != "-" else None,
            float(item['no2Value']) if item['no2Value'] != "-" else None,
            float(item['coValue']) if item['coValue'] != "-" else None,
            float(item['so2Value']) if item['so2Value'] != "-" else None
        ]
        data.append(values)

    # 결측값 처리 및 numpy 배열로 변환
    data = np.array(data)
    data = preprocess_data(data)  # 호출 추가

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 실제 값 준비 (예: 최근 10개의 pm25 값)
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # MLP 모델 예측
    with graph_mlp.as_default():
        with session_mlp.as_default():
            mlp_pred = mlp_model.predict(input_data)

    # LSTM 모델 예측
    with graph_lstm.as_default():
        with session_lstm.as_default():
            lstm_pred = lstm_model.predict(input_data)

    # GRU 모델 예측
    with graph_gru.as_default():
        with session_gru.as_default():
            gru_pred = gru_model.predict(input_data)
    # CNN 모델 예측
    with graph_cnn.as_default():
        with session_cnn.as_default():
            cnn_pred = cnn_model.predict(input_data)

    # SimpleRNN 모델 예측
    with graph_simplernn.as_default():
        with session_simplernn.as_default():
            simplernn_pred = simplernn_model.predict(input_data)
            

    # 예측 결과에서 1시간 뒤의 값을 추출
    mlp_1hour_pred = mlp_pred[0, -1]
    lstm_1hour_pred = lstm_pred[0, -1]
    gru_1hour_pred = gru_pred[0, -1]
    cnn_1hour_pred = cnn_pred[0, -1]
    simplernn_1hour_pred = simplernn_pred[0, -1]
    

    # RMSE 계산 수정
    mlp_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [mlp_1hour_pred]))
    lstm_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [lstm_1hour_pred]))
    gru_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [gru_1hour_pred]))
    cnn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [cnn_1hour_pred]))
    simplernn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [simplernn_1hour_pred]))
    fig.update_layout(annotations=[])
    # 모든 모델의 RMSE를 딕셔너리에 저장
    rmse_dict = {
        'MLP': mlp_rmse,
        'LSTM': lstm_rmse,
        'GRU': gru_rmse,
        'CNN': cnn_rmse,
        'SimpleRNN': simplernn_rmse
    }

    # 가장 낮은 RMSE를 가진 모델 찾기
    min_rmse_model = min(rmse_dict, key=rmse_dict.get)
    min_rmse_value = rmse_dict[min_rmse_model]

    # 해당 모델의 예측값 찾기
    pred_dict = {
        'MLP': mlp_1hour_pred,
        'LSTM': lstm_1hour_pred,
        'GRU': gru_1hour_pred,
        'CNN': cnn_1hour_pred,
        'SimpleRNN': simplernn_1hour_pred
    }
    min_rmse_pred = pred_dict[min_rmse_model]

    # 실제 데이터를 그래프에 추가 (점 위에 실제값 표시)
    fig.add_trace(go.Scatter(x=time_data, 
                             y=actual_data, 
                             mode='markers+lines+text', 
                             name=f"실제값 ({value})", 
                             line=dict(color='rgb(30, 144, 255)'),  # Dodger Blue
                             text=[f"{val:.3f}" for val in actual_data],
                             textposition="top center"))

    # 가장 낮은 RMSE를 가진 모델의 예측값만 그래프에 추가 (점 위에 예측값 표시)
    fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=1)], 
                             y=[min_rmse_pred], 
                             mode='markers+text', 
                             name=f"{min_rmse_model} 예측값 (RMSE: {min_rmse_value:.2f})", 
                             marker=dict(color='rgb(139, 0, 0)'),  # Dark Red
                             text=[f"{min_rmse_pred:.3f}"],
                             textposition="top center"))
    # 마지막 실제값과 예측값을 점선으로 연결
    fig.add_trace(go.Scatter(x=[time_data[-1], current_time + timedelta(hours=1)], 
                             y=[actual_data[-1], min_rmse_pred], 
                             mode='lines', 
                             line=dict(dash='dash', color='gray'),
                             showlegend=False
                             ))

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 현재 시간부터 10시간 전까지의 시간 데이터 생성
    time_data = [current_time - timedelta(hours=i) for i in range(10)]

    # 현재 시간부터 10시간 전까지의 실제 값 준비
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # 마지막 실제 값
    last_actual_value = actual_data[-1]
    
    # 실제 값의 선 색깔 정의
    actual_line_color = 'rgb(30, 144, 255)'  # Dodger Blue

    # 실제 데이터를 그래프에 추가 (선 색깔을 설정)
    #fig.add_trace(go.Scatter(x=time_data, y=actual_data, mode='markers+lines', name=f"실제값 ({value})", line=dict(color=actual_line_color)))

    return fig

# 앱 실행
if __name__ == "__main__":
    app.run_server(port=8008, host='0.0.0.0')


In [ ]:
import tensorflow as tf
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 모델 불러오기
global graph_mlp, session_mlp
graph_mlp = tf.Graph()
with graph_mlp.as_default():
    session_mlp = tf.compat.v1.Session()
    with session_mlp.as_default():
        global mlp_model
        mlp_model = load_model('./model/mlp_model.h5')

global graph_lstm, session_lstm
graph_lstm = tf.Graph()
with graph_lstm.as_default():
    session_lstm = tf.compat.v1.Session()
    with session_lstm.as_default():
        global lstm_model
        lstm_model = load_model('./model/lstm_model.h5')

global graph_gru, session_gru
graph_gru = tf.Graph()
with graph_gru.as_default():
    session_gru = tf.compat.v1.Session()
    with session_gru.as_default():
        global gru_model
        gru_model = load_model('./model/gru_model.h5')
        
# CNN 모델 불러오기
global graph_cnn, session_cnn
graph_cnn = tf.Graph()
with graph_cnn.as_default():
    session_cnn = tf.compat.v1.Session()
    with session_cnn.as_default():
        global cnn_model
        cnn_model = load_model('./model/cnn_model.h5')

# SimpleRNN 모델 불러오기
global graph_simplernn, session_simplernn
graph_simplernn = tf.Graph()
with graph_simplernn.as_default():
    session_simplernn = tf.compat.v1.Session()
    with session_simplernn.as_default():
        global simplernn_model
        simplernn_model = load_model('./model/simplernn_model.h5')


stations = [
    {'name': '종로구'},
    {'name': '강남구'},
    {'name': '중구'}
    # ... 계속 추가
]

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1(children='실시간 초미세먼지 예측 대시보드', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='station-dropdown',
        options=[{'label': station['name'], 'value': station['name']} for station in stations],
        value='중구'
    ),
    dcc.Graph(id='graph-content'),
    html.Div([
        html.P('출처: 한국환경공단, 「대기오염정보」', style={'font-size': 'small'}),
    ], style={'position': 'relative', 'bottom': '40px', 'left': '10px'}),

    dcc.Interval(id='interval-component', interval=60 * 60 * 1000, n_intervals=0),
    

    html.Div([
    html.H3('대시보드 설명', style={'padding': '3px', 'font-weight': 'bold'}),
    html.Div([
        #html.H5('주기 및 예측 범위:'),
        html.P('다양한 머신러닝 모델을 사용하여 1시간 주기로 초미세먼지 농도를 예측')
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 변수: 이산화황(SO2), 일산화탄소(CO), 오존(O3), 이산화질소(NO2), 미세먼지(PM10), 초미세먼지(PM2.5)'])
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 모델: MLP (Multi-Layer Perceptron), RNN (Recurrent Neural Network), LSTM (Long Short-Term Memory), GRU (Gated Recurrent Unit), CNN (Convolutional Neural Network)']),
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 성능 지표: RMSE (Root Mean Square Error)'])
        
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 결과 해석: 예측 성능이 가장 좋은 모델의 예측 값을 출력'])
    ], style={'margin-bottom': '5px'}),

], style={
    'padding': '20px', 
    'border': '1px solid #ddd', 
    'border-radius': '8px', 
    'margin': '20px',
    'box-shadow': '2px 2px 12px #aaa',
    'background-color': '#f9f9f9'
}) ])


def preprocess_data(data):
    for i in range(1, len(data)):
        for j in range(6):
            if data[i, j] is None and data[i-1, j] is not None:
                data[i, j] = data[i-1, j]
    
    # 스케일링
    scaler = MinMaxScaler()
    preprocessed_data = scaler.fit_transform(data)
        # NaN 값을 검사하고 0으로 대체
    preprocessed_data = np.nan_to_num(preprocessed_data)
    
    return preprocessed_data

@app.callback(
    Output('graph-content', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('station-dropdown', 'value')]
)


def update_graph(n, value):
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    current_time = datetime.now()
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    time_data = [current_time - timedelta(hours=i) for i in range(10, 0, -1)]
    # 사용자 정의 눈금 값과 레이블 생성 (시간만 표시)
    tickvals = []
    ticktext = []
    for i, time in enumerate(time_data):
        tickvals.append(time)
        ticktext.append(time.strftime("%H:%M"))  # 연도, 월, 일 제거
        
    # fig 객체 생성
    fig = go.Figure()
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="PM2.5 Concentration (μg/m³)"
    )

    # 연도, 월, 일을 상단 좌측에 표시 (예: 2023-08-21)
    fig.add_annotation(
        text=current_time.strftime("%Y-%m-%d"),
        xref="paper",
        yref="paper",
        x=0,  # 좌측 정렬
        y=1,  # 상단 정렬
        showarrow=False,
        font=dict(size=14),
      
    )
    
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
    service_key = 'xdBtiCkOLc9TIoPuquKxtR0n+Fk8awNT/qLlOCApNheoK+gweAWhIWzdUCttWKvi0xeI2bM3KCSduGfRhI922w=='


    # 선택된 측정소에 대한 데이터 수집
    params = {
        'serviceKey': service_key,
        'returnType': 'json',
        'numOfRows': '10',
        'pageNo': '1',
        'stationName': value,
        'dataTerm': 'DAILY',
        'ver': '1.0'
    }
    response = requests.get(url, params=params)
    json_object = json.loads(response.content)
    
    # 6개의 특성을 저장할 리스트
    data = []
    for item in json_object['response']['body']['items']:
        values = [
            float(item['pm10Value']) if item['pm10Value'] != "-" else None,
            float(item['pm25Value']) if item['pm25Value'] != "-" else None,
            float(item['o3Value']) if item['o3Value'] != "-" else None,
            float(item['no2Value']) if item['no2Value'] != "-" else None,
            float(item['coValue']) if item['coValue'] != "-" else None,
            float(item['so2Value']) if item['so2Value'] != "-" else None
        ]
        data.append(values)

    # 결측값 처리 및 numpy 배열로 변환
    data = np.array(data)
    data = preprocess_data(data)  # 호출 추가

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 실제 값 준비 (예: 최근 10개의 pm25 값)
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # MLP 모델 예측
    with graph_mlp.as_default():
        with session_mlp.as_default():
            mlp_pred = mlp_model.predict(input_data)

    # LSTM 모델 예측
    with graph_lstm.as_default():
        with session_lstm.as_default():
            lstm_pred = lstm_model.predict(input_data)

    # GRU 모델 예측
    with graph_gru.as_default():
        with session_gru.as_default():
            gru_pred = gru_model.predict(input_data)
    # CNN 모델 예측
    with graph_cnn.as_default():
        with session_cnn.as_default():
            cnn_pred = cnn_model.predict(input_data)

    # SimpleRNN 모델 예측
    with graph_simplernn.as_default():
        with session_simplernn.as_default():
            simplernn_pred = simplernn_model.predict(input_data)
            

    # 예측 결과에서 1시간 뒤의 값을 추출
    mlp_1hour_pred = mlp_pred[0, -1]
    lstm_1hour_pred = lstm_pred[0, -1]
    gru_1hour_pred = gru_pred[0, -1]
    cnn_1hour_pred = cnn_pred[0, -1]
    simplernn_1hour_pred = simplernn_pred[0, -1]
    

    # RMSE 계산 수정
    mlp_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [mlp_1hour_pred]))
    lstm_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [lstm_1hour_pred]))
    gru_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [gru_1hour_pred]))
    cnn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [cnn_1hour_pred]))
    simplernn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [simplernn_1hour_pred]))
    fig.update_layout(annotations=[])
    # 모든 모델의 RMSE를 딕셔너리에 저장
    rmse_dict = {
        'MLP': mlp_rmse,
        'LSTM': lstm_rmse,
        'GRU': gru_rmse,
        'CNN': cnn_rmse,
        'SimpleRNN': simplernn_rmse
    }

    # 가장 낮은 RMSE를 가진 모델 찾기
    min_rmse_model = min(rmse_dict, key=rmse_dict.get)
    min_rmse_value = rmse_dict[min_rmse_model]

    # 해당 모델의 예측값 찾기
    pred_dict = {
        'MLP': mlp_1hour_pred,
        'LSTM': lstm_1hour_pred,
        'GRU': gru_1hour_pred,
        'CNN': cnn_1hour_pred,
        'SimpleRNN': simplernn_1hour_pred
    }
    min_rmse_pred = pred_dict[min_rmse_model]

    # 실제 데이터를 그래프에 추가 (점 위에 실제값 표시)
    fig.add_trace(go.Scatter(x=time_data, 
                             y=actual_data, 
                             mode='markers+lines+text', 
                             name=f"실제값 ({value})", 
                             line=dict(color='rgb(30, 144, 255)'),  # Dodger Blue
                             text=[f"{val:.3f}" for val in actual_data],
                             textposition="top center"))



    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 현재 시간부터 10시간 전까지의 시간 데이터 생성
    time_data = [current_time - timedelta(hours=i) for i in range(10)]

    # 현재 시간부터 10시간 전까지의 실제 값 준비
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # 마지막 실제 값
    last_actual_value = actual_data[-1]
    
    # 실제 값의 선 색깔 정의
    actual_line_color = 'rgb(30, 144, 255)'  # Dodger Blue

    # 1시간, 2시간, 3시간 후 예측을 위한 빈 딕셔너리
    future_hours = [1, 2, 3]  # 이 시간들에 대한 예측을 하려고 함
    future_preds = {hour: {'value': None, 'rmse': None} for hour in future_hours}

    # 미래 예측 데이터
    for hour in future_hours:
        input_data_future = data[-10:].copy()  # 마지막 10개의 데이터로 예측을 시작
        for i in range(hour):
            with graph_mlp.as_default():
                with session_mlp.as_default():
                    mlp_pred_future = mlp_model.predict(input_data_future.reshape(1, 10, 6))
            last_pred = mlp_pred_future[0, -1]
            input_data_future = np.roll(input_data_future, shift=-1, axis=0)
            input_data_future[-1, 1] = last_pred  # PM2.5가 index 1에 있다고 가정

        rmse_future = np.sqrt(mean_squared_error([actual_data[-1]], [last_pred]))
        future_preds[hour]['value'] = last_pred
        future_preds[hour]['rmse'] = rmse_future

    # 미래 예측 데이터를 그래프에 추가
    for hour, pred_info in future_preds.items():
        pred = pred_info['value']
        rmse = pred_info['rmse']
        fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=hour)], 
                                 y=[pred], 
                                 mode='markers+text', 
                                 name=f"{min_rmse_model} {hour}-hour future prediction (RMSE: {rmse:.3f})", 
                                 marker=dict(color='rgb(139, 0, 0)'),  # Dark Red
                                 text=[f"{pred:.3f}"],
                                 textposition="top center"))    
    
    return fig

# 앱 실행
if __name__ == "__main__":
    app.run_server(port=8008, host='0.0.0.0')


# 수정Ver

In [ ]:
import tensorflow as tf
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 모델 불러오기
global graph_mlp, session_mlp
graph_mlp = tf.Graph()
with graph_mlp.as_default():
    session_mlp = tf.compat.v1.Session()
    with session_mlp.as_default():
        global mlp_model
        mlp_model = load_model('./model/mlp_model.h5')

global graph_lstm, session_lstm
graph_lstm = tf.Graph()
with graph_lstm.as_default():
    session_lstm = tf.compat.v1.Session()
    with session_lstm.as_default():
        global lstm_model
        lstm_model = load_model('./model/lstm_model.h5')

global graph_gru, session_gru
graph_gru = tf.Graph()
with graph_gru.as_default():
    session_gru = tf.compat.v1.Session()
    with session_gru.as_default():
        global gru_model
        gru_model = load_model('./model/gru_model.h5')
        
# CNN 모델 불러오기
global graph_cnn, session_cnn
graph_cnn = tf.Graph()
with graph_cnn.as_default():
    session_cnn = tf.compat.v1.Session()
    with session_cnn.as_default():
        global cnn_model
        cnn_model = load_model('./model/cnn_model.h5')

# SimpleRNN 모델 불러오기
global graph_simplernn, session_simplernn
graph_simplernn = tf.Graph()
with graph_simplernn.as_default():
    session_simplernn = tf.compat.v1.Session()
    with session_simplernn.as_default():
        global simplernn_model
        simplernn_model = load_model('./model/simplernn_model.h5')


stations = [
    {'name': '종로구'},
    {'name': '강남구'},
    {'name': '중구'}
    # ... 계속 추가
]

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1(children='실시간 초미세먼지 예측 대시보드', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='station-dropdown',
        options=[{'label': station['name'], 'value': station['name']} for station in stations],
        value='중구'
    ),
    dcc.Graph(id='graph-content'),
    html.Div([
        html.P('출처: 한국환경공단, 「대기오염정보」', style={'font-size': 'small'}),
    ], style={'position': 'relative', 'bottom': '40px', 'left': '10px'}),

    dcc.Interval(id='interval-component', interval=60 * 60 * 1000, n_intervals=0),
    

    html.Div([
    html.H3('대시보드 설명', style={'padding': '3px', 'font-weight': 'bold'}),
    html.Div([
        #html.H5('주기 및 예측 범위:'),
        html.P('다양한 머신러닝 모델을 사용하여 1시간 주기로 초미세먼지 농도를 예측')
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 변수: 이산화황(SO2), 일산화탄소(CO), 오존(O3), 이산화질소(NO2), 미세먼지(PM10), 초미세먼지(PM2.5)'])
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 모델: MLP (Multi-Layer Perceptron), RNN (Recurrent Neural Network), LSTM (Long Short-Term Memory), GRU (Gated Recurrent Unit), CNN (Convolutional Neural Network)']),
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 성능 지표: RMSE (Root Mean Square Error)'])
        
    ], style={'margin-bottom': '5px'}),

    html.Div([
        html.H5([html.Span('●', style={'font-size': 'xx-small'}),' 결과 해석: 예측 성능이 가장 좋은 모델의 예측 값을 출력'])
    ], style={'margin-bottom': '5px'}),

], style={
    'padding': '20px', 
    'border': '1px solid #ddd', 
    'border-radius': '8px', 
    'margin': '20px',
    'box-shadow': '2px 2px 12px #aaa',
    'background-color': '#f9f9f9'
}) ])


def preprocess_data(data):
    for i in range(1, len(data)):
        for j in range(6):
            if data[i, j] is None and data[i-1, j] is not None:
                data[i, j] = data[i-1, j]
    
    # 스케일링
    scaler = MinMaxScaler()
    preprocessed_data = scaler.fit_transform(data)
        # NaN 값을 검사하고 0으로 대체
    preprocessed_data = np.nan_to_num(preprocessed_data)
    
    return preprocessed_data

@app.callback(
    Output('graph-content', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('station-dropdown', 'value')]
)


def update_graph(n, value):
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    current_time = datetime.now()
    # 시간 데이터 생성 (최근 10개의 시간 단계)
    time_data = [current_time - timedelta(hours=i) for i in range(10, 0, -1)]
    # 사용자 정의 눈금 값과 레이블 생성 (시간만 표시)
    tickvals = []
    ticktext = []
    for i, time in enumerate(time_data):
        tickvals.append(time)
        ticktext.append(time.strftime("%H:%M"))  # 연도, 월, 일 제거
        
    # fig 객체 생성
    fig = go.Figure()
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="PM2.5 Concentration (μg/m³)"
    )

    # 연도, 월, 일을 상단 좌측에 표시 (예: 2023-08-21)
    fig.add_annotation(
        text=current_time.strftime("%Y-%m-%d"),
        xref="paper",
        yref="paper",
        x=0,  # 좌측 정렬
        y=1,  # 상단 정렬
        showarrow=False,
        font=dict(size=14),
      
    )
    
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
    service_key = 'xdBtiCkOLc9TIoPuquKxtR0n+Fk8awNT/qLlOCApNheoK+gweAWhIWzdUCttWKvi0xeI2bM3KCSduGfRhI922w=='


    # 선택된 측정소에 대한 데이터 수집
    params = {
        'serviceKey': service_key,
        'returnType': 'json',
        'numOfRows': '10',
        'pageNo': '1',
        'stationName': value,
        'dataTerm': 'DAILY',
        'ver': '1.0'
    }
    response = requests.get(url, params=params)
    json_object = json.loads(response.content)
    
    # 6개의 특성을 저장할 리스트
    data = []
    for item in json_object['response']['body']['items']:
        values = [
            float(item['pm10Value']) if item['pm10Value'] != "-" else None,
            float(item['pm25Value']) if item['pm25Value'] != "-" else None,
            float(item['o3Value']) if item['o3Value'] != "-" else None,
            float(item['no2Value']) if item['no2Value'] != "-" else None,
            float(item['coValue']) if item['coValue'] != "-" else None,
            float(item['so2Value']) if item['so2Value'] != "-" else None
        ]
        data.append(values)

    # 결측값 처리 및 numpy 배열로 변환
    data = np.array(data)
    data = preprocess_data(data)  # 호출 추가

    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 실제 값 준비 (예: 최근 10개의 pm25 값)
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # MLP 모델 예측
    with graph_mlp.as_default():
        with session_mlp.as_default():
            mlp_pred = mlp_model.predict(input_data)

    # LSTM 모델 예측
    with graph_lstm.as_default():
        with session_lstm.as_default():
            lstm_pred = lstm_model.predict(input_data)

    # GRU 모델 예측
    with graph_gru.as_default():
        with session_gru.as_default():
            gru_pred = gru_model.predict(input_data)
    # CNN 모델 예측
    with graph_cnn.as_default():
        with session_cnn.as_default():
            cnn_pred = cnn_model.predict(input_data)

    # SimpleRNN 모델 예측
    with graph_simplernn.as_default():
        with session_simplernn.as_default():
            simplernn_pred = simplernn_model.predict(input_data)
            

    # 예측 결과에서 1시간 뒤의 값을 추출
    mlp_1hour_pred = mlp_pred[0, -1]
    lstm_1hour_pred = lstm_pred[0, -1]
    gru_1hour_pred = gru_pred[0, -1]
    cnn_1hour_pred = cnn_pred[0, -1]
    simplernn_1hour_pred = simplernn_pred[0, -1]
    

    # RMSE 계산 수정
    mlp_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [mlp_1hour_pred]))
    lstm_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [lstm_1hour_pred]))
    gru_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [gru_1hour_pred]))
    cnn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [cnn_1hour_pred]))
    simplernn_rmse = np.sqrt(mean_squared_error([actual_data[-1]], [simplernn_1hour_pred]))
    fig.update_layout(annotations=[])
    # 모든 모델의 RMSE를 딕셔너리에 저장
    rmse_dict = {
        'MLP': mlp_rmse,
        'LSTM': lstm_rmse,
        'GRU': gru_rmse,
        'CNN': cnn_rmse,
        'SimpleRNN': simplernn_rmse
    }

    # 가장 낮은 RMSE를 가진 모델 찾기
    min_rmse_model = min(rmse_dict, key=rmse_dict.get)
    min_rmse_value = rmse_dict[min_rmse_model]

    # 해당 모델의 예측값 찾기
    pred_dict = {
        'MLP': mlp_1hour_pred,
        'LSTM': lstm_1hour_pred,
        'GRU': gru_1hour_pred,
        'CNN': cnn_1hour_pred,
        'SimpleRNN': simplernn_1hour_pred
    }
    min_rmse_pred = pred_dict[min_rmse_model]

    # 실제 데이터를 그래프에 추가 (점 위에 실제값 표시)
    fig.add_trace(go.Scatter(x=time_data, 
                             y=actual_data, 
                             mode='markers+lines+text', 
                             name=f"실제값 ({value})", 
                             line=dict(color='rgb(30, 144, 255)'),  # Dodger Blue
                             text=[f"{val:.3f}" for val in actual_data],
                             textposition="top center"))



    # 최근 10개의 시간 단계만 사용
    input_data = data[-10:]

    # 형태 확인 및 모델에 입력할 수 있는 형태로 변환
    input_data = input_data.reshape(1, 10, 6)
    # 현재 시간부터 10시간 전까지의 시간 데이터 생성
    time_data = [current_time - timedelta(hours=i) for i in range(10)]

    # 현재 시간부터 10시간 전까지의 실제 값 준비
    actual_data = data[-10:, 1]  # 두 번째 특성이 pm25 값이라고 가정
    
    # 마지막 실제 값
    last_actual_value = actual_data[-1]
    
    # 실제 값의 선 색깔 정의
    actual_line_color = 'rgb(30, 144, 255)'  # Dodger Blue

    # 1시간, 2시간, 3시간 후 예측을 위한 빈 딕셔너리
    future_hours = [1, 2, 3]  # 이 시간들에 대한 예측을 하려고 함
    future_preds = {hour: {'value': None, 'rmse': None} for hour in future_hours}

    # 미래 예측 데이터
    for hour in future_hours:
        input_data_future = data[-10:].copy()  # 마지막 10개의 데이터로 예측을 시작
        for i in range(hour):
            with graph_mlp.as_default():
                with session_mlp.as_default():
                    mlp_pred_future = mlp_model.predict(input_data_future.reshape(1, 10, 6))
            last_pred = mlp_pred_future[0, -1]
            input_data_future = np.roll(input_data_future, shift=-1, axis=0)
            input_data_future[-1, 1] = last_pred  # PM2.5가 index 1에 있다고 가정

        rmse_future = np.sqrt(mean_squared_error([actual_data[-1]], [last_pred]))
        future_preds[hour]['value'] = last_pred
        future_preds[hour]['rmse'] = rmse_future

    # 미래 예측 데이터를 그래프에 추가
    for hour, pred_info in future_preds.items():
        pred = pred_info['value']
        rmse = pred_info['rmse']
        fig.add_trace(go.Scatter(x=[current_time + timedelta(hours=hour)], 
                                 y=[pred], 
                                 mode='markers+text', 
                                 name=f"{min_rmse_model} {hour}-hour future prediction (RMSE: {rmse:.3f})", 
                                 marker=dict(color='rgb(139, 0, 0)'),  # Dark Red
                                 text=[f"{pred:.3f}"],
                                 textposition="top center"))    
    
    return fig

# 앱 실행
if __name__ == "__main__":
    app.run_server(port=8008, host='0.0.0.0')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Dash is running on http://0.0.0.0:8008/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.51:8008/ (Press CTRL+C to quit)
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET / HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET /_dash-layout HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:18:32] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:29:48] "GET / HTTP/1.1" 200 -
192.168.100.129 - - [25/Oct/2023 09:29:48] "GET /_dash-layout HTTP/1.1" 200 -
192.168.100.129 - -